Try timestamp input with time HH:MM:SS

! timedelta(days=1) = timedelta(hours=24) -> doesn't change at midnight/partial days

Would do with more time:
Would fix input to find date even if input doesn't match format

In [1]:
import pandas as pd
from datetime import timedelta, datetime

In [2]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
# accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
# groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

In [3]:
ast.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


In [4]:
# all_unique_accounts = ast.account_id.unique() # len 772-44=728

## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

### Limit Dataframe by Indicated Timestamp

In [5]:
# for all ACCOUNT_ID, return TO_STATE where STARTED_WHEN = TIMESTAMP OR nearest_preceding_date
# if ACCOUNT_ID doesn't have corresponding TIMESTAMP select earliest preceding date with record (preceding_date)
# select TO_STATE of PRECEDING_DATE and calculate time difference: TIMESTAMP-PRECEDING_DATE

In [6]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [7]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [8]:
# Change data type from string to datetime object
ast['started_when'] = pd.to_datetime(ast['started_when'])

In [9]:
# Create limited dataframe based on indicated timestamp 
df = ast.loc[ast['started_when'] <= (timestamp_dt + timedelta(hours=23,minutes=59,seconds=59))]
df.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


In [17]:
# Select max date for each unique account_id
df_grouped = df.groupby('account_id',as_index=False)['started_when'].max()
df_grouped.head(10)

,account_id,started_when
0,1,2020-01-17 13:01:20.571392
1,2,2020-02-24 02:00:21.571392
2,3,2020-02-21 11:40:47.887181
3,4,2020-02-25 04:28:59.571392
4,5,2020-02-29 20:42:56.571392
5,6,2020-02-20 18:54:47.571392
6,7,2020-02-27 21:56:59.571392
7,8,2020-02-27 05:59:01.834550
8,9,2020-02-28 09:49:07.360866
9,10,2020-02-21 15:48:49.571392


In [28]:
filtered_df = df.loc[df.groupby('account_id')['started_when'].idxmax()]
filtered_df = filtered_df[['started_when','account_id','to_state']]
filtered_df

,started_when,account_id,to_state
11863,2020-01-17 13:01:20.571392,1,DISABLED
28,2020-02-24 02:00:21.571392,2,UNLOCKED
48,2020-02-21 11:40:47.887181,3,DISABLED
67,2020-02-25 04:28:59.571392,4,UNLOCKED
85,2020-02-29 20:42:56.571392,5,DISABLED
126,2020-02-20 18:54:47.571392,6,UNLOCKED
12334,2020-02-27 21:56:59.571392,7,ENABLED
13080,2020-02-27 05:59:01.834550,8,DISABLED
162,2020-02-28 09:49:07.360866,9,DISABLED
176,2020-02-21 15:48:49.571392,10,UNLOCKED


In [11]:
df_grouped.shape
## Need to grab STATE

# for:
#     try:
#     except KeyError:
#         continue

(728, 2)

### Current State of Each Account by Indicated Timestamp

In [ ]:
# Create dataframe to show current state of each account based on indicated timestamp
df_limited2 = df_limited[['started_when','account_id','to_state']]

# Sort by account_id
df_limited2 = df_limited2.sort_values('account_id')

# Reset index
df_limited2 = df_limited2.reset_index(drop=True)
df_limited2.head()

In [ ]:
# Show in dataframe, any accounts occuring more than once during indicated timestamp

account_ids = []
df_repeat = pd.DataFrame(columns=['started_when', 'account_id', 'to_state'])

for i in range(0,len(df_limited2['account_id'])):
    account_id = df_limited2['account_id'][i]
    if account_id in account_ids:
        df_temp = df_limited2.loc[df_limited2['account_id'] == account_id]
        df_repeat = df_repeat.append(df_temp)
    else:
        account_ids.append(account_id)

df_repeat

### Total Time in State as of Indicated Timestamp

In [ ]:
# if total time = start to timestamp:
# for each account id, select state = current state for all records <= timestamp
